In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import pandas as pd
from tqdm import tqdm_notebook
from selenium.webdriver.common.keys import Keys
import traceback
import pymysql

driver = webdriver.Chrome('/Users/jiyoung/chromedriver')
driver.get('https://emart.ssg.com/category/main.ssg?dispCtgId=6000095739')
time.sleep(1.5)

loginBtn = driver.find_element_by_css_selector('#loginBtn>a') #이미지 인증방지 로그인
loginBtn.click()
time.sleep(1)

driver.switch_to.window(driver.window_handles[1]) # 로그인 새창에서 send_key
time.sleep(1)

id = driver.find_element_by_css_selector('#mem_id')
id.send_keys('dlwldud5107')

pw = driver.find_element_by_css_selector('#mem_pw')
pw.send_keys('bgbg1726!!')

login = driver.find_element_by_css_selector('.cmem_btn.cmem_btn_ornge')
login.click()
time.sleep(1)

/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/880518754.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/jiyoung/chromedriver')
/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/880518754.py:17: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  loginBtn = driver.find_element_by_css_selector('#loginBtn>a') #이미지 인증방지 로그인
/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/880518754.py:24: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  id = driver.find_element_by_css_selector('#mem_id')
/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/880518754.py:27: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_select

In [4]:
driver.switch_to.window(driver.window_handles[0]) #로그인 창 전환때문에 다시 기존창으로
driver.get('https://emart.ssg.com/category/main.ssg?dispCtgId=6000095739')
time.sleep(1)

name_list = []
image_list = []
price_list = []
review_list = []
link_list = []
star_list = []
reviewCnt_list = []
category_list = []
star_float = []
reviewCnt_int = []
price_int = []
mart_name = []
for i in tqdm_notebook(range(1,16)): 
    a = 1 #넥스트페이지 범위설정
    category_button = driver.find_element_by_css_selector('.em_nav_all>.emctg_open')
    category_button.click()
    driver.implicitly_wait(10)
    time.sleep(1)
    
    next_category = driver.find_element_by_css_selector(f'#e_gnb > div > div.em_nav_all > div > ul:nth-child(2) > li:nth-child({i}) > a')
    driver.implicitly_wait(10)
    time.sleep(1)
    
    next_category.click()
    driver.implicitly_wait(10)
    time.sleep(2)
      
    for j in tqdm_notebook(range(1,11)): #1-10페이지까지
        next_btn = driver.find_element_by_xpath(f'//*[@id="area_itemlist"]/div[2]/a[{j}]')
        next_btn.click()
        driver.implicitly_wait(10)
        time.sleep(1.5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        category = soup.select('.notranslate.clickable')[0].text.replace('\n','').replace('\t','')

        name = soup.select('.tmpl_itemlist>div>ul>li>div>div>.title>a>em.tx_ko')
        for i in name:
            name_list.append(i.text)
            mart_name.append("이마트")
            category_list.append(category)
        image = soup.select('.tmpl_itemlist>div>ul>li>div>.thmb>a>img.i1')
        for i in image:
            image_list.append("http:" + i['src'])  
        price = soup.select('.tmpl_itemlist>div>ul>li>div>div>div.opt_price>.ssg_price')
        for i in price:
            price_list.append(i.text)    
        review = driver.find_elements_by_css_selector('.tmpl_itemlist>div>ul>li>div.cunit_info>div.cunit_app')
        for i in review:
            if len(i.text) == 0:
                review_list.append(["0","0"])
            else:
                 review_list.append(i.text.replace("\n","")[3:].split('점'))    
        link = soup.select('.tmpl_itemlist>div>ul>li>div>div.thmb>a')
        for i in link:
            if "https://earlymorning" in i['href']:
                link_list.append(i['href'])
            elif "https://earlymorning" not in i['href']:
                link_list.append("https://emart.ssg.com/" + i['href'])
    print(len(link_list))
    while (a <= 3):
        a += 1
        try:
            next_page = driver.find_element_by_css_selector('a.btn_next.on')
            next_page.click()
            driver.implicitly_wait(10)
            time.sleep(2)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            category = soup.select('.notranslate.clickable')[0].text.replace('\n','').replace('\t','')
            name = soup.select('.tmpl_itemlist>div>ul>li>div>div>.title>a>em.tx_ko')
            for i in name:
                name_list.append(i.text)   
                category_list.append(category)
            image = soup.select('.tmpl_itemlist>div>ul>li>div>.thmb>a>img.i1')
            for i in image:
                image_list.append("http:" + i['src']) 
            price = soup.select('.tmpl_itemlist>div>ul>li>div>div>div.opt_price>.ssg_price')
            for i in price:
                price_list.append(i.text)    
            review = driver.find_elements_by_css_selector('.tmpl_itemlist>div>ul>li>div.cunit_info>div.cunit_app')
            for i in review:
                if len(i.text) == 0:
                    review_list.append(["0","0"])
                else:
                    review_list.append(i.text.replace("\n","")[3:].split('점'))    
            link = soup.select('.tmpl_itemlist>div>ul>li>div>div.thmb>a')
            for i in link:
                if "https://earlymorning" in i['href']:
                    link_list.append(i['href'])
                elif "https://earlymorning" not in i['href']:
                    link_list.append("https://emart.ssg.com/" + i['href'])   
        
            for i in tqdm_notebook(range(3,12)): #11페이지부터 20페이지까지
                next_btn = driver.find_element_by_xpath(f'//*[@id="area_itemlist"]/div[2]/a[{i}]')
                next_btn.click()
                driver.implicitly_wait(10)
                time.sleep(2)  

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')

                category = soup.select('.notranslate.clickable')[0].text.replace('\n','').replace('\t','')
                name = soup.select('.tmpl_itemlist>div>ul>li>div>div>.title>a>em.tx_ko')
                for i in name:
                    name_list.append(i.text)     
                    category_list.append(category)
                image = soup.select('.tmpl_itemlist>div>ul>li>div>.thmb>a>img.i1')
                for i in image:
                    image_list.append("http:" + i['src'])  
                price = soup.select('.tmpl_itemlist>div>ul>li>div>div>div.opt_price>.ssg_price')
                for i in price:
                    price_list.append(i.text)    
                review = driver.find_elements_by_css_selector('.tmpl_itemlist>div>ul>li>div.cunit_info>div.cunit_app')
                for i in review:
                    if len(i.text) == 0:
                        review_list.append(["0","0"])
                    else:
                        review_list.append(i.text.replace("\n","")[3:].split('점'))         
                link = soup.select('.tmpl_itemlist>div>ul>li>div>div.thmb>a')
                for i in link:
                    if "https://earlymorning" in i['href']:
                        link_list.append(i['href'])
                    elif "https://earlymorning" not in i['href']:
                        link_list.append("https://emart.ssg.com/" + i['href'])
        except Exception as e:
            print(traceback.format_exc())
            a = 4
            break

for j in review_list:
    star_list.append(j[0])
    reviewCnt_list.append(j[1].replace("(","").replace("개)",""))      
for i in price_list:
    price_int.append(int(i.replace(",","")))
for i in star_list:
    star_float.append(float(i))
for i in reviewCnt_list:
    reviewCnt_int.append(int(i.replace(",",""))) 
   
driver.quit() #크롤링 다끝나면 크롬종료
for i in range(len(category_list)):
    if category_list[i] == '정육/계란류':
        category_list[i] = '정육/계란'
    if category_list[i] == '수산/건해산':
        category_list[i] = '수산/건해산물'        
    if category_list[i] == '밀키트/간편식':
        category_list[i] = '냉장/냉동/밀키트'    
    if category_list[i] == '우유/유제품':
        category_list[i] = '우유/유제품/베이커리'
    if category_list[i] == '김치/반찬/델리':
        category_list[i] = '김치/반찬'
    if category_list[i] == '장류/양념/가루/오일':
        category_list[i] = '장류/양념/오일'
    if category_list[i] == '과자/간식':
        category_list[i] = '과자/간식/시리얼/떡'  
    if category_list[i] == '생수/음료/주류':
        category_list[i] = '생수/음료/커피/차/주류' 
    if category_list[i] == '커피/원두/차':
        category_list[i] = '생수/음료/커피/차/주류'    
    if category_list[i] == '면류/통조림':
        category_list[i] = '라면/통조림/즉석식품'        
    if category_list[i] == '베이커리/잼':
        category_list[i] = '우유/유제품/베이커리' 


# 크롤링 다 끝나면 MySQL에 넣어주기    
conn = pymysql.connect(host = 'project-db-stu.ddns.net', port = 3307, user = 'seocho_0830_1', password = 'smhrd1', db = 'seocho_0830_1', charset = 'utf8')
cur = conn.cursor(pymysql.cursors.DictCursor)

#cur.execute("drop table product;")
#conn.commit()
#cur.execute("CREATE TABLE product (prodCode INT NOT NULL AUTO_INCREDENT primary key, martName VARCHAR(50), categoryName VARCHAR(255), prodName VARCHAR(255), prodPrice INT, prodImage VARCHAR(255), link VARCHAR(255), prodStar float, prodReview INT);")
#conn.commit()

for i in tqdm_notebook(range(48000)):
    cur.execute(f"insert into emart values(null,'{mart_name[i]}','{category_list[i]}','{name_list[i]}',{price_int[i]},'{image_list[i]}','{link_list[i]}',{star_float[i]},{reviewCnt_int[i]});")
    conn.commit()
conn.close()

/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/1899987413.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1,17)):


  0%|          | 0/16 [00:00<?, ?it/s]

/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/1899987413.py:19: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  category_button = driver.find_element_by_css_selector('.em_nav_all>.emctg_open')
/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/1899987413.py:24: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  next_category = driver.find_element_by_css_selector(f'#e_gnb > div > div.em_nav_all > div > ul:nth-child(2) > li:nth-child({i}) > a')
/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/1899987413.py:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j in tqdm_notebook(range(1,11)): #1-10페이지까지


  0%|          | 0/10 [00:00<?, ?it/s]

/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/1899987413.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  next_btn = driver.find_element_by_xpath(f'//*[@id="area_itemlist"]/div[2]/a[{j}]')
/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/1899987413.py:54: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  review = driver.find_elements_by_css_selector('.tmpl_itemlist>div>ul>li>div.cunit_info>div.cunit_app')


...

48800


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_7622/1899987413.py", line 103, in <cell line: 17>
    next_btn = driver.find_element_by_xpath(f'//*[@id="area_itemlist"]/div[2]/a[{i}]')
  File "/Users/jiyoung/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 526, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "/Users/jiyoung/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 1251, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "/Users/jiyoung/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 430, in execute
    self.error_handler.check_response(response)
  File "/Users/jiyoung/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/remote/errorhandler.py", line 247, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.e

IndexError: list index out of range

In [5]:
len(name_list), len(image_list), len(price_int), len(star_float), len(reviewCnt_int), len(link_list), len(category_list)

(50291, 50291, 50291, 50291, 50291, 50291, 50291)

### MySQL 연결

In [259]:
# mySQL 연결
import pymysql

conn = pymysql.connect(host = '127.0.0.1', user = 'root', password = '1234', db = 'emart', charset = 'utf8')
cur = conn.cursor(pymysql.cursors.DictCursor)
                   
for i in range(50000):
    cur.execute(f"insert into product values({i},'{category_list[i]}','{name_list[i]}',{price_int[i]},'{image_list[i]}','{link_list[i]}',{star_float[i]},{reviewCnt_int[i]});")
    conn.commit()
conn.close()

In [258]:
# 실시간이어야하니까
cur.execute("drop table product;")
conn.commit()
cur.execute("CREATE TABLE product (prodCode INT NOT NULL, prodName VARCHAR(255), prodImage VARCHAR(255), prodPrice int, prodStar float, prodReview int, link VARCHAR(255), categoryName VARCHAR(255));")
conn.commit()

for i in range(50000):
    cur.execute(f"insert into product values({i},'{category_list[i]}','{name_list[i]}',{price_int[i]},'{image_list[i]}','{link_list[i]}',{star_float[i]},{reviewCnt_int[i]});")
    conn.commit()
conn.close()

InterfaceError: (0, '')

### 엑셀 저장

In [6]:
# 데이터프레임
data = {'카테고리별':category_list, '상품명':name_list, '가격':price_list, '이미지':image_list, '별점':star_list, '리뷰':reviewCnt_list, '링크':link_list}
df = pd.DataFrame(data)
df.columns = ['카테고리별','상품명','가격','이미지','별점','리뷰','링크']
df

,카테고리별,상품명,가격,이미지,별점,리뷰,링크
0,과일,[국내산] 생생대추 (80g),"5,980",http://sitem.ssgcdn.com/19/91/01/item/00000080...,4.91,658,https://emart.ssg.com//item/itemView.ssg?itemI...
1,과일,[미국산] 냉동 타트체리 500g,"8,280",http://sitem.ssgcdn.com/83/01/08/item/10002890...,4.67,61,https://emart.ssg.com//item/itemView.ssg?itemI...
2,과일,[국내산] 냉동오디 500g,"11,300",http://sitem.ssgcdn.com/76/88/15/item/10000051...,4.78,433,https://emart.ssg.com//item/itemView.ssg?itemI...
3,과일,[국산과일 먹는날 최대 ~28%할인] 캠벨포도/샤인머스켓/복숭아/메론,"16,800",http://sitem.ssgcdn.com/81/03/43/item/10002874...,4.6,"30,394",https://emart.ssg.com//item/dealItemView.ssg?i...
4,과일,제철과일 행사 [무화과/햇사과外],"11,900",http://sitem.ssgcdn.com/73/00/48/item/10001894...,4.69,"45,119",https://emart.ssg.com//item/dealItemView.ssg?i...
...,...,...,...,...,...,...,...
50286,친환경/유기농,[누림]천년숨결 홍삼정 240g,"184,000",http://sitem.ssgcdn.com/24/04/63/item/10000056...,0,0,https://emart.ssg.com//item/itemView.ssg?itemI...
50287,친환경/유기농,오스트리아 샬크뮐레 유기농 홍화씨 오일 250ml X 6병 수입완제품,"229,800",http://sitem.ssgcdn.com/85/92/39/item/10000343...,0,0,https://emart.ssg.com//item/itemView.ssg?itemI...
50288,친환경/유기농,하와이 유기농 노니주스 100% 착즙원액 X 6병 (세달분) 수입완제품,"270,000",http://sitem.ssgcdn.com/61/11/55/item/10000325...,0,0,https://emart.ssg.com//item/itemView.ssg?itemI...
50289,친환경/유기농,[22/08/26일 순차출고][냉장] 설성목장 무행생제 한우 명품 1호세트 2kg,"300,000",http://sitem.ssgcdn.com/95/70/44/item/10000354...,5,3,https://emart.ssg.com//item/itemView.ssg?itemI...


In [7]:
# 엑셀로 저장
data = pd.ExcelWriter(r'emart.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(data)
data.close()

/var/folders/wy/34j5dqtj7rq_z64p1mtz2f_w0000gn/T/ipykernel_805/1792230818.py:2: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  data = pd.ExcelWriter(r'emart.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})


### 리뷰를 평점/리뷰개수로 나누기

In [230]:
# 리뷰를 평점,리뷰개수로 나누기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

review_list = []
star_list = []
reviewCnt_list = []

review = driver.find_elements_by_css_selector('.tmpl_itemlist>div>ul>li>div.cunit_info>div.cunit_app')
for i in review:
    if len(i.text) == 0:
        review_list.append(["0","0"])
    else:
         review_list.append(i.text.replace("\n","")[3:].split('점'))    

for j in review_list:
    star_list.append(j[0])
    reviewCnt_list.append(j[1].replace("(","").replace("개)",""))
    
print(len(star_list), len(reviewCnt_list))


80 80


### 문자열 -> 정수형 변환

In [241]:
# 문자열을 정수형으로 바꿔주기
star_float = []
reviewCnt_int = []
price_int = []

for i in price_list:
    price_int.append(int(i.replace(",","")))
for i in star_list:
    star_float.append(float(i))
for i in reviewCnt_list:
    reviewCnt_int.append(int(i.replace(",","")))   